### Imports

In [1]:
import string, numpy as np
from random import shuffle
from keras.layers import Dense
from keras.layers.recurrent import GRU
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


### Tokenizing Methods

In [2]:
tokenizer = Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts(string.printable[:-2])

def to_vec(text):
    if isinstance(text, str): text = [text]
    tokenized = np.array(tokenizer.texts_to_sequences(list(text)))
    return tokenized.reshape(*tokenized.shape, 1)

def to_txt(vector):
    vector = vector.reshape(vector.shape[1]).tolist()
    return tokenizer.sequences_to_texts([map(round, vector)])[0].replace(' ', '')

### Generating and Preparing Data

In [3]:
def encrypt(string):
    length = len(string)
    return ''.join([chr(ord(string[(i+1)%length]) + 17) for i in range(length)])

text = list(map(str, range(10000, 99999)))
shuffle(text)

train_cipher = to_vec(map(encrypt, text))
train_text = to_vec(text)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(GRU(128, activation='relu', input_shape=train_cipher.shape[1:], return_sequences=True))
model.add(GRU(64, activation='relu', return_sequences=True))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, batch_size=10, epochs=5)

Epoch 1/5
89999/89999 [==============================] - 72s 799us/step - loss: 1.5098 - acc: 0.8005
Epoch 2/5
89999/89999 [==============================] - 76s 840us/step - loss: 1.3570 - acc: 0.8217
Epoch 3/5
89999/89999 [==============================] - 78s 870us/step - loss: 1.3536 - acc: 0.8220
Epoch 4/5
89999/89999 [==============================] - 85s 948us/step - loss: 1.3508 - acc: 0.8222
Epoch 5/5
89999/89999 [==============================] - 74s 820us/step - loss: 1.3488 - acc: 0.8223


### Predicting and Evaluating the Model

In [5]:
test_cyhper = 'AFGBH'
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: 50561
Test cipher vs encrypted prediction: AFGBH vs AFGBF
